In [1]:
import pandas as pd
import csv
import numpy as np
from datetime import datetime
import dateutil.parser
from IPython.display import display
import math
import copy 

import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split



pd.options.display.max_columns = None
pd.set_option('display.max_rows', None)



In [ ]:
#can skip to way later if using Jing Chun's preprocessed csv

In [21]:
df=pd.read_csv("/Users/joonghochoi/Desktop/NFLBigDataBowl/NFL/combined_per_player_block.csv")
df.head(3)

,Unnamed: 0,gameId,playId,frameId,nflId,x.x,y.x,dist,dist_to_qb,area,dir,force,x_force,y_force,pff_role
0,1,2021090900,1078,1,37082,15.33,33.05,3.145100,4.455581,94.9405,266.94,128.000002,127.817497,-6.832851,Pass Block
1,2,2021090900,1078,1,42654,15.35,26.90,3.025854,4.279685,94.9405,330.79,3.200000,1.561638,2.793078,Pass Block
2,3,2021090900,1078,1,46119,15.76,31.50,1.592674,3.891786,94.9405,288.55,381.439991,361.622834,121.348230,Pass Block


In [22]:
df1=pd.read_csv("/Users/joonghochoi/Desktop/NFLBigDataBowl/NFL/combined_per_player_rush.csv")
df1.head(3)

,Unnamed: 0,gameId,playId,frameId,nflId,x.x,y.x,dist,dist_to_qb,area,dir,force,x_force,y_force,pff_role
0,1,2021090900,1078,1,34777,17.26,31.84,1.579377,5.403739,85.2136,0.90,43.400000,-0.681698,43.394646,Pass Rush
1,2,2021090900,1078,1,35454,17.29,35.06,4.795798,7.262981,85.2136,269.25,16.500000,16.498586,-0.215978,Pass Rush
2,3,2021090900,1078,1,41915,17.53,24.72,5.547097,7.361963,85.2136,305.98,200.000003,161.844404,117.500596,Pass Rush


In [83]:
print(set(df.pff_role))
print(set(df1.pff_role))
print(df.shape)
print(df1.shape)
print(df2.shape)

{'Pass Block'}
{'Pass Rush'}
(243576, 15)
(180877, 15)
(1118122, 21)


In [27]:
frames = [df,df1]

result = pd.concat(frames)
print(result.shape)
result.head(3)

(424453, 15)


,Unnamed: 0,gameId,playId,frameId,nflId,x.x,y.x,dist,dist_to_qb,area,dir,force,x_force,y_force,pff_role
0,1,2021090900,1078,1,37082,15.33,33.05,3.145100,4.455581,94.9405,266.94,128.000002,127.817497,-6.832851,Pass Block
1,2,2021090900,1078,1,42654,15.35,26.90,3.025854,4.279685,94.9405,330.79,3.200000,1.561638,2.793078,Pass Block
2,3,2021090900,1078,1,46119,15.76,31.50,1.592674,3.891786,94.9405,288.55,381.439991,361.622834,121.348230,Pass Block


In [29]:
result=result.drop(columns=['Unnamed: 0'])
result.head(3)

,gameId,playId,frameId,nflId,x.x,y.x,dist,dist_to_qb,area,dir,force,x_force,y_force,pff_role
0,2021090900,1078,1,37082,15.33,33.05,3.145100,4.455581,94.9405,266.94,128.000002,127.817497,-6.832851,Pass Block
1,2021090900,1078,1,42654,15.35,26.90,3.025854,4.279685,94.9405,330.79,3.200000,1.561638,2.793078,Pass Block
2,2021090900,1078,1,46119,15.76,31.50,1.592674,3.891786,94.9405,288.55,381.439991,361.622834,121.348230,Pass Block


In [34]:
print(set(result.pff_role))


{'Pass Rush', 'Pass Block'}


In [84]:
df2=pd.read_csv("/Users/joonghochoi/Desktop/NFLBigDataBowl/NFL/2023/pffScoutingData.csv")
set(df2.playId)

,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
0,2021090900,97,25511,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021090900,97,35481,Pass Route,TE-L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021090900,97,35634,Pass Route,LWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
df3=df2[df2.columns[df2.columns.isin(['gameId','playId','nflId','pff_hit','pff_hurry','pff_sack'])]]
df3.head(3)

,gameId,playId,nflId,pff_hit,pff_hurry,pff_sack
0,2021090900,97,25511,NaN,NaN,NaN
1,2021090900,97,35481,NaN,NaN,NaN
2,2021090900,97,35634,NaN,NaN,NaN


In [87]:
print(result.shape)
print(df3.shape)

(424453, 14)
(188254, 6)


In [91]:
df4=df3.merge(result, left_on=['gameId','playId','nflId'],right_on=['gameId','playId','nflId'])

In [93]:
print(df4.shape)
df4.head(2)

(424453, 17)


,gameId,playId,nflId,pff_hit,pff_hurry,pff_sack,frameId,x.x,y.x,dist,dist_to_qb,area,dir,force,x_force,y_force,pff_role
0,2021090900,97,40151,NaN,NaN,NaN,1,42.10,24.02,0.521145,4.466453,90.3112,308.34,631.620006,495.407066,391.810759,Pass Block
1,2021090900,97,40151,NaN,NaN,NaN,10,40.82,24.55,0.563368,4.500589,44.2454,285.24,175.450004,169.279999,46.119254,Pass Block


In [96]:
df5=df4.dropna()
print(df5.shape)
df5.head(2)

(180877, 17)


,gameId,playId,nflId,pff_hit,pff_hurry,pff_sack,frameId,x.x,y.x,dist,dist_to_qb,area,dir,force,x_force,y_force,pff_role
33,2021090900,97,41263,0.0,1.0,0.0,1,43.30,18.89,6.935151,7.802083,271.4125,288.76,238.499994,225.829433,76.702765,Pass Rush
34,2021090900,97,41263,0.0,1.0,0.0,10,41.42,19.90,5.793641,6.604150,0.0000,310.65,363.050001,275.447190,236.504014,Pass Rush


In [107]:
df5['outcome']=0
df5.loc[df5['pff_hurry'] == 1, 'outcome']='Hurry'
df5.loc[df5['pff_hit'] == 1, 'outcome']='Hit'
df5.loc[df5['pff_sack'] == 1, 'outcome']='Sack'

<ipython-input-107-f56a258df7df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['outcome']=0


In [109]:
df5.head(2)

,gameId,playId,nflId,pff_hit,pff_hurry,pff_sack,frameId,x.x,y.x,dist,dist_to_qb,area,dir,force,x_force,y_force,pff_role,outcome
33,2021090900,97,41263,0.0,1.0,0.0,1,43.30,18.89,6.935151,7.802083,271.4125,288.76,238.499994,225.829433,76.702765,Pass Rush,Hurry
34,2021090900,97,41263,0.0,1.0,0.0,10,41.42,19.90,5.793641,6.604150,0.0000,310.65,363.050001,275.447190,236.504014,Pass Rush,Hurry


In [110]:
x_df=df5[df5.columns[~df5.columns.isin(['gameId','playId','nflId','outcome'])]]
x_df.head(3)

,pff_hit,pff_hurry,pff_sack,frameId,x.x,y.x,dist,dist_to_qb,area,dir,force,x_force,y_force,pff_role,outcome
33,0.0,1.0,0.0,1,43.30,18.89,6.935151,7.802083,271.4125,288.76,238.499994,225.829433,76.702765,Pass Rush,Hurry
34,0.0,1.0,0.0,10,41.42,19.90,5.793641,6.604150,0.0000,310.65,363.050001,275.447190,236.504014,Pass Rush,Hurry
35,0.0,1.0,0.0,11,41.18,20.13,5.511346,6.447271,0.0000,314.12,347.149985,249.213209,241.673102,Pass Rush,Hurry


In [116]:
y_df=df5[df5.columns[df5.columns.isin(['outcome'])]]
y_df.head(3)

,outcome
33,Hurry
34,Hurry
35,Hurry


In [ ]:
#continue from here using Jing chun's.

In [2]:
new_df=pd.read_csv("/Users/joonghochoi/Desktop/NFLBigDataBowl/NFL/2023/by_frame_all.csv")
new_df.head(2)

,Unnamed: 0,playId,gameId.x,frameId,dist_name,dist_to_qb_name,area_name,x_force_name,y_force_name,norm_weighted_x_name,norm_weighted_y_name,net_x_force_bottom_name,net_x_force_middle_name,net_x_force_top_name,net_y_force_bottom_name,net_y_force_middle_name,net_y_force_y_name,gameId.y,hit,hurry,sack,bad_thing
0,1,55,2021091300,5,2.997692,4.280639,47.856033,-350.675972,18.667865,-336.460695,33.720778,0.0,-597.356840,-122.274350,0.0,-48.033561,281.792595,2021103112,0,1,1,1
1,2,55,2021091300,3,2.964995,4.353640,51.280733,-407.128155,38.829978,-392.110675,53.648174,0.0,60.448704,-86.818662,0.0,-11.572894,-38.766470,2021103112,0,1,1,1


In [3]:
#new_df[new_df.bad_thing==0,]

In [4]:
new_df1=new_df.drop(columns=['Unnamed: 0','playId','gameId.x','gameId.y','hit','hurry','sack'])
print(new_df1.shape)


(36665, 15)


In [5]:
new_df1.head(2)

,frameId,dist_name,dist_to_qb_name,area_name,x_force_name,y_force_name,norm_weighted_x_name,norm_weighted_y_name,net_x_force_bottom_name,net_x_force_middle_name,net_x_force_top_name,net_y_force_bottom_name,net_y_force_middle_name,net_y_force_y_name,bad_thing
0,5,2.997692,4.280639,47.856033,-350.675972,18.667865,-336.460695,33.720778,0.0,-597.356840,-122.274350,0.0,-48.033561,281.792595,1
1,3,2.964995,4.353640,51.280733,-407.128155,38.829978,-392.110675,53.648174,0.0,60.448704,-86.818662,0.0,-11.572894,-38.766470,1


In [6]:
x_df=new_df1[new_df1.columns[~new_df1.columns.isin(['bad_thing'])]]
x_df.head(3)


,frameId,dist_name,dist_to_qb_name,area_name,x_force_name,y_force_name,norm_weighted_x_name,norm_weighted_y_name,net_x_force_bottom_name,net_x_force_middle_name,net_x_force_top_name,net_y_force_bottom_name,net_y_force_middle_name,net_y_force_y_name
0,5,2.997692,4.280639,47.856033,-350.675972,18.667865,-336.460695,33.720778,0.0,-597.356840,-122.274350,0.0,-48.033561,281.792595
1,3,2.964995,4.353640,51.280733,-407.128155,38.829978,-392.110675,53.648174,0.0,60.448704,-86.818662,0.0,-11.572894,-38.766470
2,2,2.952259,4.371644,608.395633,-310.765476,39.107485,-298.388494,52.769109,0.0,2.063553,-36.320075,0.0,-6.810917,-32.231495


In [7]:
y_df=new_df1[new_df1.columns[new_df1.columns.isin(['bad_thing'])]]
y_df.head(3)

,bad_thing
0,1
1,1
2,1


In [8]:
bad=y_df.loc[y_df['bad_thing'] == 1]

not_bad=y_df.loc[y_df['bad_thing'] == 0]


print(bad.shape)
print(not_bad.shape) #imbalanced as expected

(26626, 1)
(10039, 1)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.33, random_state=42)

In [10]:
X_train.head(2)
y_train.head(2)

,bad_thing
32269,1
3181,1


In [11]:
clf = xgb.XGBClassifier(max_depth=5, n_estimators=30,objective='binary:logistic')
# clf = xgb.XGBClassifier(max_depth=5, objective='multi:softprob', n_estimators=1000, 
#                         num_classes=2)

clf.fit(X_train, y_train)  
pred = clf.predict(X_test)

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[22:38:21] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [12]:

print("Accuracy of Model::",accuracy_score(y_test,pred))



Accuracy of Model:: 0.7378512396694215


In [33]:
#check before feature engineering 

new_df2=copy.deepcopy(new_df1)
new_df2['net_xy_force_middle_ratio']=new_df2['net_x_force_middle_name']/new_df2['net_y_force_middle_name']
new_df2['net_xy_force_top_ratio']=new_df2['net_x_force_top_name']/new_df2['net_y_force_y_name']
new_df2['dist_to_qb_ratio_area']=new_df2['dist_to_qb_name']/new_df2['area_name']
new_df2.head(2)

,frameId,dist_name,dist_to_qb_name,area_name,x_force_name,y_force_name,norm_weighted_x_name,norm_weighted_y_name,net_x_force_bottom_name,net_x_force_middle_name,net_x_force_top_name,net_y_force_bottom_name,net_y_force_middle_name,net_y_force_y_name,bad_thing,net_xy_force_middle_ratio,net_xy_force_top_ratio,dist_to_qb_ratio_area
0,5,2.997692,4.280639,47.856033,-350.675972,18.667865,-336.460695,33.720778,0.0,-597.356840,-122.274350,0.0,-48.033561,281.792595,1,12.436239,-0.433916,0.089448
1,3,2.964995,4.353640,51.280733,-407.128155,38.829978,-392.110675,53.648174,0.0,60.448704,-86.818662,0.0,-11.572894,-38.766470,1,-5.223300,2.239530,0.084898


In [34]:

new_df2.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows with NaN
new_df2.dropna(inplace=True)
print(new_df2.isnull().values.any())


False


In [35]:
x_df1=new_df2[new_df2.columns[~new_df2.columns.isin(['bad_thing'])]]
y_df1=new_df2[new_df2.columns[new_df2.columns.isin(['bad_thing'])]]

In [36]:
count = np.isinf(new_df2).values.sum()
print("Infinity values... ",count)

Infinity values...  0


In [37]:
X_train, X_test, y_train, y_test = train_test_split(x_df1, y_df1, test_size=0.33, random_state=42)

In [38]:
# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
# print(y_test.shape)

# print(X_train.isnull().values.any())
# print(X_test.isnull().values.any())
# print(y_train.isnull().values.any())
# print(y_test.isnull().values.any())

(18401, 17)
(9064, 17)
(18401, 1)
(9064, 1)


In [40]:
clf = xgb.XGBClassifier(max_depth=5, n_estimators=30,objective='binary:logistic')
# clf = xgb.XGBClassifier(max_depth=5, objective='multi:softprob', n_estimators=1000, 
#                         num_classes=2)

clf.fit(X_train, y_train)  
pred = clf.predict(X_test)

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[22:47:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [41]:
print("Accuracy of Model::",accuracy_score(y_test,pred))

#didn't perform better

Accuracy of Model:: 0.7220873786407767


In [48]:
#grid search
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score

parameters = {
              'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [4,5,6],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [30,40,50] #number of trees, change it to 1000 for better results
            }

In [49]:
skf = StratifiedKFold(n_splits=4, shuffle = True, random_state = 42)

random_search = RandomizedSearchCV(xgb.XGBClassifier(), param_distributions=parameters, scoring='roc_auc', cv=skf.split(x_df1, y_df1), verbose=3, random_state=42 )

In [50]:
random_search.fit(x_df1, y_df1)


/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Fitting 4 folds for each of 9 candidates, totalling 36 fits
[23:00:09] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/4] END colsample_bytree=0.7, learning_rate=0.05, max_depth=4, n_estimators=30, objective=binary:logistic, subsample=0.8;, score=0.615 total time=   0.3s
[23:00:10] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 2/4] END colsample_bytree=0.7, learning_rate=0.05, max_depth=4, n_estimators=30, objective=binary:logistic, subsample=0.8;, score=0.610 total time=   0.3s
[23:00:10] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

[CV 3/4] END colsample_bytree=0.7, learning_rate=0.05, max_depth=6, n_estimators=50, objective=binary:logistic, subsample=0.8;, score=0.668 total time=   1.1s
[23:00:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 4/4] END colsample_bytree=0.7, learning_rate=0.05, max_depth=6, n_estimators=50, objective=binary:logistic, subsample=0.8;, score=0.670 total time=   1.1s
[23:00:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x1388bc740>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=na...
                                           random_state=None, reg_alpha=None,
                                           reg_lambda=None,
                                           scale_pos_weight=None,
                                           subsample=None, tr

In [52]:
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best hyperparameters:')
print(random_search.best_params_)


 Best estimator:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=50, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

 Best hyperparameters:
{'subsample': 0.8, 'objective': 'binary:logistic', 'n_estimators': 50, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.7}

 best score
0.6691320771733934


In [54]:
clf = xgb.XGBClassifier(max_depth=6,subsample=0.8, n_estimators=50,objective='binary:logistic',learning_rate=0.05,colsample_bytree=0.7)
# clf = xgb.XGBClassifier(max_depth=5, objective='multi:softprob', n_estimators=1000, 
#                         num_classes=2)

clf.fit(X_train, y_train)  
pred = clf.predict(X_test)
print("Accuracy of Model::",accuracy_score(y_test,pred))



[23:04:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy of Model:: 0.7127096204766108
